In [1]:
import pandas as pd

In [3]:
# 2015-2016
df0 = pd.read_sas("00_DIQ_I.XPT")
df1 = pd.read_sas("10_DEMO_I.XPT")
df2 = pd.read_sas("20_BMX_I.XPT")
df3 = pd.read_sas("30_GHB_I.XPT")
df4 = pd.read_sas("40_HDL_I.XPT")
df41 = pd.read_sas("41_TRIGLY_I.XPT")
df5 = pd.read_sas("50_BPX_I.XPT")
# 2017-2018
df0_ = pd.read_sas("00_DIQ_J.XPT")
df1_ = pd.read_sas("10_DEMO_J.XPT")
df2_ = pd.read_sas("20_BMX_J.XPT")
df3_ = pd.read_sas("30_GHB_J.XPT")
df4_ = pd.read_sas("40_HDL_J.XPT")
df41_ = pd.read_sas("41_TRIGLY_J.XPT")
df5_ = pd.read_sas("50_BPX_J.XPT")

In [26]:
# 2015-2016 merge by 'SEQN'
df0_1 = df0[['SEQN','DIQ010', 'DIQ172']].copy()
df1_1 = df1[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDEXPRG']].copy()
df2_1 = df2[['SEQN', 'BMXBMI']].copy()
df4_1 = df4[['SEQN', 'LBDHDD']].copy()
df41_1 = df41[['SEQN', 'LBDLDL', 'LBXTR']].copy()
df5_1 = df5[['SEQN', 'BPXSY1', 'BPXDI1', 'BPXSY2', 'BPXDI2', 'BPXSY3', 'BPXDI3']].copy()
merged_df = df0_1.merge(df1_1, on='SEQN', how = 'inner')
merged_df = merged_df.merge(df2_1, on='SEQN', how = 'inner')
merged_df = merged_df.merge(df3, on='SEQN', how = 'inner')
merged_df = merged_df.merge(df4_1, on='SEQN', how = 'inner')
merged_df = merged_df.merge(df41_1, on='SEQN', how = 'inner')
merged_df = merged_df.merge(df5_1, on='SEQN', how = 'inner')
# 2017-2018 merge by 'SEQN'
df0_1_ = df0_[['SEQN', 'DIQ010', 'DIQ172']].copy()
df1_1_ = df1_[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDEXPRG']].copy()
df2_1_ = df2_[['SEQN', 'BMXBMI']].copy()
df4_1_ = df4_[['SEQN', 'LBDHDD']].copy()
df41_1_ = df41_[['SEQN', 'LBDLDL', 'LBXTR']].copy()
df5_1_ = df5_[['SEQN', 'BPXSY1', 'BPXDI1', 'BPXSY2', 'BPXDI2', 'BPXSY3', 'BPXDI3']].copy()
merged_df_ = df0_1_.merge(df1_1_, on='SEQN', how = 'inner')
merged_df_ = merged_df_.merge(df2_1_, on='SEQN', how = 'inner')
merged_df_ = merged_df_.merge(df3_, on='SEQN', how = 'inner')
merged_df_ = merged_df_.merge(df4_1_, on='SEQN', how = 'inner')
merged_df_ = merged_df_.merge(df41_1_, on='SEQN', how = 'inner')
merged_df_ = merged_df_.merge(df5_1_, on='SEQN', how = 'inner')
# 2015-2016 & 2017-2018 merge
df = pd.concat([merged_df, merged_df_], axis=0)

In [27]:
# Delete pregnancy and drop the column
df = df[df['RIDEXPRG'] != 1]
df = df.drop(columns=['RIDEXPRG'])

# Delete NA
df = df.dropna(how='all')

# Filter Age>18
df = df[df['RIDAGEYR'] > 18]

In [28]:
df.shape

(5195, 16)

In [29]:
# Control Group 0 -> has diabetes: Glycohemoglobin (%) > 5.6 & BMI > 25
HasDiabetes = (df['LBXGH'] > 5.6) & (df['BMXBMI'] > 25)
count0 = df[HasDiabetes].shape[0]
percent0 = count0/df.shape[0]
print(count0)
print(percent0)

1774
0.34148219441770933


In [30]:
# Variable Group 1 -> Learn and evaluate on diabetes + cholesterol data only
# HDL < 40 | LDL > 130 | Triglyceride > 150
HasHighCholesterol = (df['LBXGH'] > 5.6) & (df['BMXBMI'] > 25) & ((df['LBDHDD'] < 40) | (df['LBDLDL'] > 130) | (df['LBXTR'] > 150))
count1 = df[HasHighCholesterol].shape[0]
percent1 = count1/df.shape[0]
print(count1)
print(percent1)

929
0.17882579403272378


In [31]:
# Variable Group 2 -> Learn and evaluate on diabetes + blood pressure data only
# Caculate mean of blood pressure
Systolic_columns = df[['BPXSY1', 'BPXSY2', 'BPXSY3']]
Diastolic_columns = df[['BPXDI1', 'BPXDI2', 'BPXDI3']]
df['SystolicMean'] = Systolic_columns.mean(axis=1)
df['DiastolicMean'] = Diastolic_columns.mean(axis=1)

HasHighBloodPressure = (df['LBXGH'] > 5.6) & (df['BMXBMI'] > 25) & ((df['SystolicMean'] > 120) | (df['DiastolicMean'] > 80))
count2 = df[HasHighBloodPressure].shape[0]
percent2 = count2/df.shape[0]
print(count2)
print(percent2)

1252
0.2410009624639076


In [32]:
# Variable Group #3 (Learn and evaluate on diabetes + cholesterol + blood pressure data)
HasBoth = HasHighCholesterol & HasHighBloodPressure
count3 = df[HasBoth].shape[0]
percent3 = count3/df.shape[0]
print(count3)
print(percent3)

677
0.13031761308950915
